In [32]:
import numpy as np
import pandas as pd
from pandas_datareader import data
import datetime as dt

In [9]:
def load_financial_data(name, output_file):
    try:
        df = pd.read_pickle(output_file)
        print('File data found...reading',name,'data')
    except FileNotFoundError:
        print('File not found...downloading', name, 'data')
        df = data.DataReader(name, 'yahoo', '2001-01-01', '2019-11-24')
        df.to_pickle(output_file)
    return df

In [10]:
SP500 = load_financial_data('^GSPC', '^GSPC_data.pkl')

File not found...downloading ^GSPC data


In [55]:
def get_logreturn(yahoo_dataframe):
    out = pd.DataFrame(index = yahoo_dataframe.index)
    prices = yahoo_dataframe.Close
    out['LogReturn'] = np.log(prices.shift(1, 'd')/prices)/1
    np.sqrt( (out.index.shift(1, 'd')-out.index).days )
    return out

In [56]:
get_logreturn(SP500)

,LogReturn
Date,
2001-01-02,NaN
2001-01-03,-0.048884
2001-01-04,0.010609
2001-01-05,0.026593
2001-01-08,NaN
...,...
2019-11-18,NaN
2019-11-19,0.000593
2019-11-20,0.003763


In [70]:
a=SP500.index.shift(1, 'd')

b=SP500.index

SP500.index.shift(1,'d')-SP500.index

In [73]:
a-b

TimedeltaIndex(['1 days', '1 days', '1 days', '1 days', '1 days', '1 days',
                '1 days', '1 days', '1 days', '1 days',
                ...
                '1 days', '1 days', '1 days', '1 days', '1 days', '1 days',
                '1 days', '1 days', '1 days', '1 days'],
               dtype='timedelta64[ns]', name='Date', length=4754, freq=None)

In [74]:
a

DatetimeIndex(['2001-01-03', '2001-01-04', '2001-01-05', '2001-01-06',
               '2001-01-09', '2001-01-10', '2001-01-11', '2001-01-12',
               '2001-01-13', '2001-01-17',
               ...
               '2019-11-12', '2019-11-13', '2019-11-14', '2019-11-15',
               '2019-11-16', '2019-11-19', '2019-11-20', '2019-11-21',
               '2019-11-22', '2019-11-23'],
              dtype='datetime64[ns]', name='Date', length=4754, freq=None)

In [75]:
b

DatetimeIndex(['2001-01-02', '2001-01-03', '2001-01-04', '2001-01-05',
               '2001-01-08', '2001-01-09', '2001-01-10', '2001-01-11',
               '2001-01-12', '2001-01-16',
               ...
               '2019-11-11', '2019-11-12', '2019-11-13', '2019-11-14',
               '2019-11-15', '2019-11-18', '2019-11-19', '2019-11-20',
               '2019-11-21', '2019-11-22'],
              dtype='datetime64[ns]', name='Date', length=4754, freq=None)